<a href="https://colab.research.google.com/github/navalepratham18/LLM-Powered-Pandas-Query-Engine/blob/main/Float_Chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Convertion from netCDF to CSV



In [ ]:
import xarray as xr

# --- 1. SET YOUR FILENAMES HERE ---
netcdf_file = '/content/drive/MyDrive/data/tempsal.nc'   # Change this to your NetCDF filename
csv_file = '/content/drive/MyDrive/data/output_file.csv'         # Change this to your desired output CSV filename

try:
    # --- 2. OPEN THE NETCDF FILE ---
    # xarray opens the file and loads the data into a Dataset object.
    print(f"Opening NetCDF file: {netcdf_file}...")
    dataset = xr.open_dataset(netcdf_file)
    print("File opened successfully.")

    # --- 3. CONVERT TO A PANDAS DATAFRAME ---
    # .to_dataframe() intelligently flattens the multi-dimensional data
    # into a 2D table format suitable for CSV.
    print("Converting to DataFrame...")
    df = dataset.to_dataframe()
    print("Conversion successful.")

    # Optional: Reset the index if the coordinates are part of the index
    # This makes columns like 'time', 'latitude', 'longitude' regular columns.
    df = df.reset_index()

    # --- 4. SAVE TO CSV ---
    # .to_csv() saves the DataFrame to the specified file.
    # index=False prevents pandas from writing the DataFrame index as a column.
    print(f"Saving data to CSV file: {csv_file}...")
    df.to_csv(csv_file, index=False)
    print("Successfully converted NetCDF to CSV!")

except FileNotFoundError:
    print(f"Error: The file '{netcdf_file}' was not found. Please make sure it's in the same directory as the script.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Opening NetCDF file: /content/drive/MyDrive/data/tempsal.nc...
File opened successfully.
Converting to DataFrame...
Conversion successful.
Saving data to CSV file: /content/drive/MyDrive/data/output_file.csv...


KeyboardInterrupt: 

## csv to postgreSQL

In [ ]:
# --- 1. INSTALL LIBRARIES ---
print("Installing libraries...")
%pip install pandas sqlalchemy psycopg2-binary &> /dev/null
print("Installation complete.")

# --- 2. MOUNT GOOGLE DRIVE & SETUP ---
import os
import pandas as pd
from sqlalchemy import create_engine, text
from google.colab import userdata, drive

# Mount your Google Drive
drive.mount('/content/drive')
print("Google Drive mounted.")

# Securely import the database connection string
try:
    DATABASE_URL = userdata.get('DB_CONNECTION_STRING')
except userdata.SecretNotFoundError as e:
    print("Secret not found. Please add your DB_CONNECTION_STRING from Neon to Colab's secrets.")
    raise e
print("Database connection string loaded successfully.")


# --- 3. SETUP DATABASE CONNECTION AND TABLE (WITH CORRECTIONS) ---
TABLE_NAME = "observations"
DATA_DIR = "/content/drive/MyDrive/data"
engine = create_engine(DATABASE_URL)

# --- THIS IS THE KEY FIX ---
# The new schema now includes a 'float_id' column.
create_table_sql = f"""
DROP TABLE IF EXISTS {TABLE_NAME}; -- Drop the old table to start fresh
CREATE TABLE {TABLE_NAME} (
    id SERIAL PRIMARY KEY,
    float_id INTEGER, -- Added this crucial column
    observation_time TIMESTAMP,
    latitude REAL,
    longitude REAL,
    depth REAL,
    temperature REAL,
    temperature_error REAL,
    salinity REAL,
    salinity_error REAL
);
"""
# Create the new, correct table
with engine.connect() as connection:
    print(f"Creating a new, correct table named '{TABLE_NAME}'...")
    connection.execute(text("COMMIT;"))
    connection.execute(text(create_table_sql))
    connection.execute(text("COMMIT;"))
    print("Table created successfully.")


# --- 4. PROCESS AND INSERT THE FIRST 100,000 ROWS ---
csv_file_to_process = [f for f in os.listdir(DATA_DIR) if f.endswith('.csv')][0]
print(f"\n--- Processing first 100,000 rows from: {csv_file_to_process} ---")
df = pd.read_csv(os.path.join(DATA_DIR, csv_file_to_process), nrows=100000)

# 1. Transform and rename columns
rename_dict = { "TAXIS": "observation_time", "YAXIS": "latitude", "XAXIS": "longitude", "ZAX": "depth", "TEMP": "temperature", "TERR": "temperature_error", "SAL": "salinity", "SERR": "salinity_error"}
df.rename(columns=rename_dict, inplace=True)

# 2. Extract float_id from the original data
# We assume the original CSV has a column like 'ad_observation_id' or similar to get the float_id
# If your CSV doesn't have it, we'll need to adjust. For now, we assume it does.
# Let's check for a column that contains an underscore, which is likely the one.
id_col = [col for col in df.columns if '_' in str(col)]
if id_col:
    # Split the original ID column to create a proper 'float_id'
    df['float_id'] = df[id_col[0]].str.split('_', expand=True)[0]
else:
    # If no such column is found, we'll assign a placeholder ID
    df['float_id'] = 0
    print("Warning: Could not find a source column for 'float_id'. Using placeholder.")


# 3. Select the final columns for the database, including float_id
required_cols = ['float_id', 'observation_time', 'latitude', 'longitude', 'depth', 'temperature', 'temperature_error', 'salinity', 'salinity_error']
cols_to_insert = [col for col in required_cols if col in df.columns]
df_to_insert = df[cols_to_insert]

if 'observation_time' in df_to_insert.columns:
    df_to_insert['observation_time'] = pd.to_datetime(df_to_insert['observation_time'])
df_to_insert.dropna(inplace=True)
df_to_insert['float_id'] = pd.to_numeric(df_to_insert['float_id'], errors='coerce')


# 4. Load the data into the new table
df_to_insert.to_sql(TABLE_NAME, engine, if_exists='append', index=False, method='multi')
print(f"Successfully inserted the sample data into the new database table.")

# --- 5. VERIFICATION ---
with engine.connect() as connection:
    result = connection.execute(text(f"SELECT COUNT(*) FROM {TABLE_NAME};")).scalar_one()
    print(f"\nVerification complete. The '{TABLE_NAME}' table now contains {result} total rows.")
    sample_df = pd.read_sql(f"SELECT * FROM {TABLE_NAME} LIMIT 5", connection)
    print("\nHere are the first 5 rows, which now include 'float_id':")
    print(sample_df)

Installing libraries...
Installation complete.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted.
Database connection string loaded successfully.
Creating a new, correct table named 'observations'...
Table created successfully.

--- Processing first 100,000 rows from: output_file.csv ---


/tmp/ipython-input-131200344.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_insert['observation_time'] = pd.to_datetime(df_to_insert['observation_time'])
/tmp/ipython-input-131200344.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_insert.dropna(inplace=True)
/tmp/ipython-input-131200344.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

Successfully inserted the sample data into the new database table.

Verification complete. The 'observations' table now contains 66015 total rows.

Here are the first 5 rows, which now include 'float_id':
   id float_id observation_time  latitude  longitude  depth  temperature  \
0   1     None       2010-01-10     -30.0       31.0    5.0       25.940   
1   2     None       2010-01-10     -30.0       32.0    5.0       25.841   
2   3     None       2010-01-10     -30.0       33.0    5.0       25.712   
3   4     None       2010-01-10     -30.0       34.0    5.0       25.533   
4   5     None       2010-01-10     -30.0       35.0    5.0       25.300   

   temperature_error  salinity  salinity_error  
0                0.7    35.343             0.7  
1                0.6    35.350             0.6  
2                0.6    35.357             0.6  
3                0.5    35.365             0.5  
4                0.4    35.373             0.4  


## connection of vector database

In [ ]:
# --- 1. INSTALL LIBRARIES ---
print("Installing libraries for the vector database...")
%pip install chromadb pandas sqlalchemy psycopg2-binary sentence-transformers &> /dev/null
print("Installation complete.")

# --- 2. MOUNT GOOGLE DRIVE & SETUP ---
import os
import shutil
import pandas as pd
from sqlalchemy import create_engine, text
from google.colab import userdata, drive
import chromadb
from sentence_transformers import SentenceTransformer

# Mount your Google Drive to access your files
drive.mount('/content/drive')
print("Google Drive mounted.")

# Securely import the database connection string from Colab's secrets manager
try:
    DATABASE_URL = userdata.get('DB_CONNECTION_STRING')
except userdata.SecretNotFoundError as e:
    print("Secret not found. Please add your DB_CONNECTION_STRING from Neon to Colab's secrets.")
    raise e
print("Database connection string loaded successfully.")

# --- 3. DEFINE STORAGE PATHS ---
# We will build the DB in Colab's temporary storage first...
CHROMA_PATH_LOCAL = "./chroma_db_temp"
# ...and then save the final result to Google Drive.
CHROMA_PATH_DRIVE = "/content/drive/MyDrive/data/chroma_db"

# --- 4. SETUP CHROMA VECTOR DATABASE & EMBEDDING MODEL ---
# Load the local embedding model
print("\nLoading local embedding model (BAAI/bge-small-en-v1.5)...")
embedding_model = SentenceTransformer('BAAI/bge-small-en-v1.5')
print("Embedding model loaded.")

# --- 5. LOAD OR BUILD THE VECTOR DATABASE ---
# Check if the database has already been built and saved in Google Drive
if os.path.exists(CHROMA_PATH_DRIVE):
    print(f"Found existing database in Google Drive. Copying to local session for use...")
    # Copy the saved DB from Drive to the local temp directory
    shutil.copytree(CHROMA_PATH_DRIVE, CHROMA_PATH_LOCAL)
    # Connect to the local copy
    client = chromadb.PersistentClient(path=CHROMA_PATH_LOCAL)
    collection = client.get_collection(name="argo_profiles")
    print("Successfully loaded database for use.")
else:
    print("No existing database found in Google Drive. Building a new one...")

    # --- a) Connect to PostgreSQL and Fetch Data ---
    print(f"Connecting to PostgreSQL to fetch source data...")
    engine = create_engine(DATABASE_URL)
    with engine.connect() as connection:
        df = pd.read_sql("SELECT * FROM observations", connection)
    print(f"Successfully fetched {len(df)} rows from PostgreSQL.")

    # --- b) Prepare Data for Summarization ---
    print("Grouping data into individual profiles...")
    df['observation_date_only'] = pd.to_datetime(df['observation_time']).dt.date
    profiles = df.groupby(['float_id', 'observation_date_only'])
    print(f"Found {len(profiles)} unique profiles to summarize.")

    # --- c) Setup a fresh local ChromaDB client ---
    client = chromadb.PersistentClient(path=CHROMA_PATH_LOCAL)
    collection = client.get_or_create_collection(name="argo_profiles")
    print(f"ChromaDB collection 'argo_profiles' created in local temp storage.")

    # --- d) Generate Summaries and Store in Local DB ---
    print("Generating summaries, vectorizing, and storing locally...")
    documents_to_add = []
    metadatas_to_add = []
    ids_to_add = []
    for (float_id, obs_date), profile_df in profiles:
        summary_text = (
            f"A profile from ARGO float number {int(float_id)} on {obs_date.strftime('%Y-%m-%d')}. "
            f"It contains {len(profile_df)} measurements from various depths."
        )
        documents_to_add.append(summary_text)
        metadatas_to_add.append({"float_id": int(float_id), "date": str(obs_date)})
        ids_to_add.append(f"profile_{int(float_id)}_{obs_date}")

    if documents_to_add:
        collection.add(documents=documents_to_add, metadatas=metadatas_to_add, ids=ids_to_add)
        print("Successfully added all profiles to the local vector database!")

    # --- e) Save the final database to Google Drive ---
    print(f"\nSaving the completed database to your Google Drive at {CHROMA_PATH_DRIVE}...")
    shutil.copytree(CHROMA_PATH_LOCAL, CHROMA_PATH_DRIVE)
    print("Database successfully saved to Google Drive for future use.")

# --- 6. VERIFICATION ---
count = collection.count()
print(f"\nVerification complete. The 'argo_profiles' collection now contains {count} entries.")

print("\nHere is an example entry from the vector database:")
example = collection.peek()
print(example)

Installing libraries for the vector database...
Installation complete.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted.
Database connection string loaded successfully.

Loading local embedding model (BAAI/bge-small-en-v1.5)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding model loaded.
Found existing database in Google Drive. Copying to local session for use...
Successfully loaded database for use.

Verification complete. The 'argo_profiles' collection now contains 0 entries.

Here is an example entry from the vector database:
{'ids': [], 'embeddings': array([], dtype=float64), 'documents': [], 'uris': None, 'included': ['metadatas', 'documents', 'embeddings'], 'data': None, 'metadatas': []}


## Integration of LLM-Dataset ~RAG

In [ ]:
# --- 1. INSTALL LIBRARIES ---
print("Installing all necessary libraries...")
%pip install llama-cpp-python --force-reinstall --upgrade --no-cache-dir
%pip install llama-index llama-index-llms-llama-cpp llama-index-embeddings-huggingface pandas sqlalchemy psycopg2-binary &> /dev/null
%pip install llama-index-vector-stores-chroma chromadb sentence-transformers &> /dev/null
print("Installation complete.")

# --- 2. SETUP & IMPORTS ---
import os
from google.colab import drive
from sqlalchemy import create_engine, text
from llama_index.core import Settings, SQLDatabase, VectorStoreIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine import RouterQueryEngine, NLSQLTableQueryEngine
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

# Mount Google Drive
drive.mount('/content/drive')
print("Google Drive mounted.")

# --- 3. CONFIGURE THE 100% LOCAL AI MODELS ---

# --- LLM (The "Brain"): Fell-ssm-360m-chat ---
# Using an ultra-lightweight model to prevent RAM crashes in Colab
print("Configuring local LLM (Fell-ssm-360m-chat)...")
llm = LlamaCPP(
    model_url='https://huggingface.co/matthieuvincent/Fell-ssm-360m-chat-GGUF/resolve/main/Fell-ssm-360m-chat-Q4_K_M.gguf',
    temperature=0.1,
    max_new_tokens=256,
    context_window=2048,
    generate_kwargs={},
    model_kwargs={"n_gpu_layers": -1},
    verbose=True,
)

# --- Embedding Model (The "Indexer"): BGE ---
print("Setting up local embedding model...")
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.llm = llm
print("Local LLM and embedding model configured successfully.")


# --- 4. CONNECT TO DATABASES & CREATE QUERY ENGINES ("THE ASSISTANTS") ---

# --- Assistant 1: The Data Analyst (PostgreSQL) ---
# We still need the DB connection string from secrets for the SQL database
try:
    DATABASE_URL = userdata.get('DB_CONNECTION_STRING')
except userdata.SecretNotFoundError as e:
    print("Secret not found. Please add your DB_CONNECTION_STRING to Colab's secrets.")
    raise e
print("\nConnecting to PostgreSQL database...")
engine = create_engine(DATABASE_URL)
sql_database = SQLDatabase(engine, include_tables=["observations"])
print("Connection to PostgreSQL successful.")
sql_query_engine = NLSQLTableQueryEngine(sql_database=sql_database, tables=["observations"])

# --- Assistant 2: The Digital Librarian (ChromaDB) ---
print("\nLoading vector database from Google Drive...")
CHROMA_PATH_DRIVE = "/content/drive/MyDrive/data/chroma_db"
db = chromadb.PersistentClient(path=CHROMA_PATH_DRIVE)
chroma_collection = db.get_or_create_collection("argo_profiles")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
vector_index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
print("Vector database is ready.")
vector_query_engine = vector_index.as_query_engine()


# --- 5. CREATE THE ROUTER QUERY ENGINE ("THE MANAGER") ---
print("\nCreating the Router (The Manager)...")
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    name="sql_database_analyzer",
    description="Use this tool for precise, factual, or mathematical questions about ARGO ocean data, such as temperature, salinity, and depth."
)
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    name="semantic_profile_searcher",
    description="Use this tool for general, open-ended, or summary-style questions about ARGO profiles based on concepts like location or time."
)
query_engine = RouterQueryEngine.from_defaults(
    query_engine_tools=[sql_tool, vector_tool],
    llm=Settings.llm,
    verbose=True
)
print("RAG Query Pipeline is ready!")


# --- 6. ASK QUESTIONS! ---
print("\n--- Start Chatting With Your Data ---")
while True:
    try:
        query = input("Ask a question about the ARGO data (or type 'exit' to quit): ")
        if query.lower() == 'exit':
            break
        print("\nThinking...")
        response = query_engine.query(query)
        print("\nResponse:")
        print(str(response))
        print("\n" + "="*50 + "\n")
    except EOFError:
        break

Installing all necessary libraries...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 97.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 272.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 286.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 234.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 222.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 274.0 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl size=4503285 sha256=2add16f8b22bd00f3a88b5b2a252bf3a8fa061a7a01ff7cf923216a3b60a6c51
  Stored in directory: /tmp/pip-ephem-wheel-cache-g4lwk2ha/wheels/90/82/ab/8784ee3fb99ddb07fd36a679ddbe63122cc07718f

Installation complete.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted.
Configuring local LLM (Fell-ssm-360m-chat)...
Error downloading model: ('Content should be at least 1 MB, but is only', '29', 'bytes')
Download incomplete. Removing partially downloaded file.


ValueError: Download incomplete.

## Using LlamaIndex

In [ ]:
# --- 1. INSTALL LIBRARIES ---
print("Installing all necessary libraries...")
# We need pgvector, the google genai integrations, and sentence-transformers
%pip install llama-index llama-index-vector-stores-postgres llama-index-llms-google-genai llama-index-embeddings-google-genai pandas sqlalchemy psycopg2-binary &> /dev/null
%pip install pgvector sentence-transformers &> /dev/null
%pip install nest_asyncio &> /dev/null
print("Installation complete.")

# --- 2. SETUP & IMPORTS ---
import os
import nest_asyncio
nest_asyncio.apply()

import pandas as pd
from google.colab import userdata, drive
from sqlalchemy import create_engine, text, make_url
from llama_index.core import Settings, VectorStoreIndex, Document
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from llama_index.vector_stores.postgres import PGVectorStore

# Mount Google Drive
drive.mount('/content/drive')
print("Google Drive mounted.")

# Securely import API key and DB connection string
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    DATABASE_URL = userdata.get('DB_CONNECTION_STRING')
except userdata.SecretNotFoundError as e:
    print("Secret not found. Please ensure GOOGLE_API_KEY and DB_CONNECTION_STRING are in Colab's secrets.")
    raise e
print("API Key and DB connection string loaded successfully.")

# --- 3. CONFIGURE THE LLM AND EMBEDDING MODEL ---
# LLM (The "Brain") for generating answers
Settings.llm = GoogleGenAI(model_name="models/gemini-1.5-pro-latest")
print("LLM configured to use Gemini 1.5 Pro.")

# --- THIS IS THE FIX ---
# Embedding Model (The "Indexer") for vectorization
Settings.embed_model = GoogleGenAIEmbedding(model_name="models/embedding-001")
print("Embedding model configured to use Google's embedding model.")


# --- 4. CONNECT TO POSTGRESQL & PREPARE DATABASE ---
TABLE_NAME = "observations"
VECTOR_TABLE_NAME = "argo_vector_store"
engine = create_engine(DATABASE_URL)
url = make_url(DATABASE_URL)

with engine.connect() as connection:
    # Enable the pgvector extension
    connection.execute(text("CREATE EXTENSION IF NOT EXISTS vector;"))
    connection.commit()
    print("PostgreSQL and pgvector are ready.")

# --- 5. SETUP THE PGVectorStore AND BUILD THE INDEX (IF NEEDED) ---
print("\nSetting up the RAG pipeline with PostgreSQL...")

# This is the LlamaIndex component that connects directly to your PostgreSQL database
vector_store = PGVectorStore.from_params(
    database=url.database,
    host=url.host,
    password=url.password,
    port=url.port,
    user=url.username,
    table_name=VECTOR_TABLE_NAME,
    embed_dim=768  # Google's embedding model dimension
)

# Check if the vector store is empty. If so, build it.
with engine.connect() as connection:
    try:
        count_result = connection.execute(text(f"SELECT COUNT(*) FROM {VECTOR_TABLE_NAME};")).scalar_one()
    except Exception:
        count_result = 0

if count_result > 0:
    print(f"Vector store already contains {count_result} entries. Loading index.")
    index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
else:
    print("Vector store is empty. Building index for the first time...")
    # --- a) Fetch data from the observations table ---
    with engine.connect() as connection:
        df = pd.read_sql(f"SELECT * FROM {TABLE_NAME}", connection)
    print(f"Fetched {len(df)} rows from PostgreSQL to create summaries.")

    # --- b) Create high-level summaries (Documents) ---
    df['observation_date_only'] = pd.to_datetime(df['observation_time']).dt.date
    profiles = df.groupby(['float_id', 'observation_date_only'])
    documents = []
    for (float_id, obs_date), profile_df in profiles:
        summary_text = (
            f"A profile from ARGO float number {int(float_id)} on {obs_date.strftime('%Y-%m-%d')}. "
            f"It contains {len(profile_df)} measurements from various depths. "
            f"The latitude was approximately {profile_df['latitude'].mean():.2f} and longitude was {profile_df['longitude'].mean():.2f}."
        )
        documents.append(Document(text=summary_text))

    # --- c) Create the index, which automatically embeds and stores the documents ---
    index = VectorStoreIndex.from_documents(documents, vector_store=vector_store, show_progress=True)
    print("Successfully built and stored the index in PostgreSQL.")

print("RAG pipeline setup complete.")

# --- 6. CREATE THE QUERY ENGINE ---
print("\nCreating the query engine...")
query_engine = index.as_query_engine(streaming=True, similarity_top_k=5)
print("Query engine is ready!")

# --- 7. ASK QUESTIONS! ---
print("\n--- Start Chatting With Your ARGO Data ---")
while True:
    try:
        query = input("Ask a question about the ARGO data (or type 'exit' to quit): ")
        if query.lower() == 'exit':
            break
        print("\nThinking...")
        response = query_engine.query(query)
        response.print_response_stream()
        print("\n" + "="*50 + "\n")
    except EOFError:
        break

Installing all necessary libraries...
Installation complete.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted.
API Key and DB connection string loaded successfully.
LLM configured to use Gemini 1.5 Pro.
Embedding model configured to use Google's embedding model.
PostgreSQL and pgvector are ready.

Setting up the RAG pipeline with PostgreSQL...
Vector store is empty. Building index for the first time...
Fetched 66015 rows from PostgreSQL to create summaries.


Parsing nodes: 0it [00:00, ?it/s]

Successfully built and stored the index in PostgreSQL.
RAG pipeline setup complete.

Creating the query engine...
Query engine is ready!

--- Start Chatting With Your ARGO Data ---
Ask a question about the ARGO data (or type 'exit' to quit): Tell me about the profile from float 6908 on January 10, 2010.

Thinking...


ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.\n* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0\n* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0\n* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0\n* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/embed_content_free_tier_requests', 'quotaId': 'EmbedContentRequestsPerDayPerUserPerProjectPerModel-FreeTier'}, {'quotaMetric': 'generativelanguage.googleapis

## with local embedded llm

In [ ]:
# --- 1. INSTALL LIBRARIES ---
print("Installing all necessary libraries...")
# We now use the huggingface embeddings library instead of the google one
%pip install llama-index llama-index-llms-google-genai llama-index-embeddings-huggingface pandas sqlalchemy psycopg2-binary &> /dev/null
%pip install pgvector sentence-transformers &> /dev/null
%pip install nest_asyncio &> /dev/null
print("Installation complete.")

# --- 2. SETUP & IMPORTS ---
import os
import nest_asyncio
nest_asyncio.apply()

import pandas as pd
from google.colab import userdata, drive
from sqlalchemy import create_engine, text, make_url
from llama_index.core import Settings, VectorStoreIndex, Document
from llama_index.llms.google_genai import GoogleGenAI
# --- THIS IS THE KEY CHANGE ---
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.postgres import PGVectorStore

# Mount Google Drive
drive.mount('/content/drive')
print("Google Drive mounted.")

# Securely import API key and DB connection string
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    DATABASE_URL = userdata.get('DB_CONNECTION_STRING')
except userdata.SecretNotFoundError as e:
    print("Secret not found. Please ensure GOOGLE_API_KEY and DB_CONNECTION_STRING are in Colab's secrets.")
    raise e
print("API Key and DB connection string loaded successfully.")

# --- 3. CONFIGURE THE LLM AND EMBEDDING MODEL ---
# LLM (The "Brain") for generating answers - STILL USES GEMINI API
Settings.llm = GoogleGenAI(model_name="models/gemini-1.5-pro-latest")
print("LLM configured to use Gemini 1.5 Pro.")

# --- THIS IS THE FIX ---
# Embedding Model (The "Indexer") now runs locally. No more API calls for this.
print("Setting up local embedding model... This may download the model for the first time.")
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
print("Local embedding model configured successfully.")


# --- 4. CONNECT TO POSTGRESQL & PREPARE DATABASE ---
TABLE_NAME = "observations"
VECTOR_TABLE_NAME = "argo_vector_store"
engine = create_engine(DATABASE_URL)
url = make_url(DATABASE_URL)

with engine.connect() as connection:
    # Enable the pgvector extension
    connection.execute(text("CREATE EXTENSION IF NOT EXISTS vector;"))
    connection.commit()
    print("PostgreSQL and pgvector are ready.")

# --- 5. SETUP THE PGVectorStore AND BUILD THE INDEX (IF NEEDED) ---
print("\nSetting up the RAG pipeline with PostgreSQL...")

# This connects to your PostgreSQL vector table.
vector_store = PGVectorStore.from_params(
    database=url.database,
    host=url.host,
    password=url.password,
    port=url.port,
    user=url.username,
    table_name=VECTOR_TABLE_NAME,
    embed_dim=384  # The dimension for BAAI/bge-small-en-v1.5 is 384
)

# Check if the vector store is empty. If so, build it.
with engine.connect() as connection:
    try:
        # We need to create the table if it doesn't exist to check its count
        connection.execute(text(f"CREATE TABLE IF NOT EXISTS {VECTOR_TABLE_NAME} (id uuid PRIMARY KEY, embedding vector(384), metadata json, text text);"))
        connection.commit()
        count_result = connection.execute(text(f"SELECT COUNT(*) FROM {VECTOR_TABLE_NAME};")).scalar_one()
    except Exception:
        count_result = 0

if count_result > 0:
    print(f"Vector store already contains {count_result} entries. Loading index.")
    index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
else:
    print("Vector store is empty. Building index for the first time...")
    with engine.connect() as connection:
        df = pd.read_sql(f"SELECT * FROM {TABLE_NAME}", connection)
    print(f"Fetched {len(df)} rows from PostgreSQL to create summaries.")

    df['observation_date_only'] = pd.to_datetime(df['observation_time']).dt.date
    profiles = df.groupby(['float_id', 'observation_date_only'])
    documents = []
    for (float_id, obs_date), profile_df in profiles:
        summary_text = (
            f"A profile from ARGO float number {int(float_id)} on {obs_date.strftime('%Y-%m-%d')}. "
            f"It contains {len(profile_df)} measurements from various depths. "
            f"The latitude was approximately {profile_df['latitude'].mean():.2f} and longitude was {profile_df['longitude'].mean():.2f}."
        )
        documents.append(Document(text=summary_text))

    # This now uses the LOCAL embedding model. No API calls!
    index = VectorStoreIndex.from_documents(documents, vector_store=vector_store, show_progress=True)
    print("Successfully built and stored the index in PostgreSQL using local embeddings.")

print("RAG pipeline setup complete.")

# --- 6. CREATE THE QUERY ENGINE ---
print("\nCreating the query engine...")
query_engine = index.as_query_engine(streaming=True, similarity_top_k=5)
print("Query engine is ready!")

# --- 7. ASK QUESTIONS! ---
print("\n--- Start Chatting With Your ARGO Data ---")
while True:
    try:
        query = input("Ask a question about the ARGO data (or type 'exit' to quit): ")
        if query.lower() == 'exit':
            break
        print("\nThinking...")
        response = query_engine.query(query)
        response.print_response_stream()
        print("\n" + "="*50 + "\n")
    except EOFError:
        break

Installing all necessary libraries...
Installation complete.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted.
API Key and DB connection string loaded successfully.
LLM configured to use Gemini 1.5 Pro.
Setting up local embedding model... This may download the model for the first time.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Local embedding model configured successfully.
PostgreSQL and pgvector are ready.

Setting up the RAG pipeline with PostgreSQL...
Vector store is empty. Building index for the first time...
Fetched 66015 rows from PostgreSQL to create summaries.


Parsing nodes: 0it [00:00, ?it/s]

Successfully built and stored the index in PostgreSQL using local embeddings.
RAG pipeline setup complete.

Creating the query engine...
Query engine is ready!

--- Start Chatting With Your ARGO Data ---

Thinking...
Empty Response


Thinking...
Empty Response


Thinking...
Empty Response


Thinking...
Empty Response

Ask a question about the ARGO data (or type 'exit' to quit): exit


## pandas Query Engine

In [ ]:
# --- 1. INSTALL LIBRARIES ---
# print("Installing all necessary libraries...")
# We need the experimental library for the PandasQueryEngine
%pip install llama-index llama-index-llms-google-genai pandas openpyxl &> /dev/null
%pip install llama-index-experimental &> /dev/null
%pip install nest_asyncio &> /dev/null
print("Installation complete.")

# --- 2. SETUP & IMPORTS ---
import os
import nest_asyncio
nest_asyncio.apply()

import pandas as pd
from google.colab import userdata, drive
from llama_index.core import Settings
from llama_index.llms.google_genai import GoogleGenAI
# The correct tool for this job: PandasQueryEngine
from llama_index.experimental.query_engine import PandasQueryEngine

# Mount Google Drive
drive.mount('/content/drive')
print("Google Drive mounted.")

# Securely import API key
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
except userdata.SecretNotFoundError as e:
    print("Secret not found. Please ensure GOOGLE_API_KEY is in Colab's secrets.")
    raise e
print("API Key loaded successfully.")

# --- 3. CONFIGURE THE LLM ---
Settings.llm = GoogleGenAI(model_name="models/gemini-1.5-pro-latest")
print("LLM configured to use Gemini 1.5 Pro.")

# --- 4. LOAD THE CSV FILE INTO A PANDAS DATAFRAME ---
# This is much simpler. We just load the data into a table.
DATA_PATH = "/content/drive/MyDrive/data/observations.csv"
# print(f"\nLoading data from {DATA_PATH} into a DataFrame...")
df = pd.read_csv(DATA_PATH)
# print(f"Successfully loaded DataFrame with {len(df)} rows.")

# --- 5. CREATE AND RUN THE QUERY ENGINE ---
print("\n--- Start Chatting With Your Data ---")
while True:
    try:
        query = input("Ask a question about the ARGO data (or type 'exit' to quit): ")
        if query.lower() == 'exit':
            print("Exiting. Goodbye!")
            break

        # We create a fresh, stateless query engine for each question
        # print("\nCreating a fresh Pandas Query Engine...")
        query_engine = PandasQueryEngine(df=df, llm=Settings.llm, verbose=True)

        print("Thinking...")
        response = query_engine.query(query)

        print("\nResponse:")
        print(str(response))
        print("\n" + "="*50 + "\n")
    except EOFError:
        print("\nExiting due to input interruption.")
        break

Installation complete.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted.
API Key loaded successfully.
LLM configured to use Gemini 1.5 Pro.

--- Start Chatting With Your Data ---
Ask a question about the ARGO data (or type 'exit' to quit): What was the maximum temperature recorded on January 10, 2010, at a depth of 5 meters?
Thinking...
> Pandas Instructions:
```
df.loc[(df['observation_time'] == '2010-01-10 00:00:00') & (df['depth'] == 5), 'temperature'].max()

```
> Pandas Output: 25.94

Response:
25.94


Ask a question about the ARGO data (or type 'exit' to quit): What was the salinity for the observation with id 5?
Thinking...
> Pandas Instructions:
```
df.query('id == 5')['salinity'].values[0]

```
> Pandas Output: 35.373

Response:
35.373


Ask a question about the ARGO data (or type 'exit' to quit): exit
Exiting. Goodbye!


# Questions :
1. What was the maximum temperature recorded on January 10, 2010, at a depth of 5 meters?
2. What was the salinity for the observation with id 5?
3. What was the maximum temperature recorded on January 10, 2010, at a depth of 5 meters?
4.

## with NLP

In [ ]:
# --- 1. INSTALL LIBRARIES ---
print("Installing all necessary libraries...")
%pip install llama-index llama-index-llms-google-genai pandas openpyxl &> /dev/null
%pip install llama-index-experimental &> /dev/null
%pip install nest_asyncio &> /dev/null
print("Installation complete.")

# --- 2. SETUP & IMPORTS ---
import os
import nest_asyncio
nest_asyncio.apply()

import pandas as pd
from google.colab import userdata, drive
from llama_index.core import Settings
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.experimental.query_engine import PandasQueryEngine

# Mount Google Drive
drive.mount('/content/drive')
print("Google Drive mounted.")

# Securely import API key
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
except userdata.SecretNotFoundError as e:
    print("Secret not found. Please ensure GOOGLE_API_KEY is in Colab's secrets.")
    raise e
print("API Key loaded successfully.")

# --- 3. CONFIGURE THE LLM ---
Settings.llm = GoogleGenAI(model_name="models/gemini-1.5-pro-latest")
print("LLM configured to use Gemini 1.5 Pro.")

# --- 4. LOAD THE CSV FILE INTO A PANDAS DATAFRAME ---
DATA_PATH = "/content/drive/MyDrive/data/output_file.csv"
print(f"\nLoading data from {DATA_PATH} into a DataFrame...")
df = pd.read_csv(DATA_PATH)
print(f"Successfully loaded DataFrame with {len(df)} rows.")


# --- 5. CREATE AND RUN THE QUERY ENGINE ---
print("\n--- Start Chatting With Your Data ---")

# --- THIS IS THE NEW INSTRUCTION FOR THE LLM ---
instruction_str = (
    "1. After the query has been run, do not just return the raw numerical or string output.\n"
    "2. Synthesize a final, user-friendly answer in a complete, natural language sentence."
)

while True:
    try:
        query = input("Ask a question about the ARGO data (or type 'exit' to quit): ")
        if query.lower() == 'exit':
            print("Exiting. Goodbye!")
            break

        print("\nCreating a fresh Pandas Query Engine...")
        # We add the new instruction here
        query_engine = PandasQueryEngine(
            df=df,
            llm=Settings.llm,
            verbose=True,
            instruction_str=instruction_str
        )

        print("Thinking...")
        response = query_engine.query(query)

        print("\nResponse:")
        print(str(response))
        print("\n" + "="*50 + "\n")
    except EOFError:
        print("\nExiting due to input interruption.")
        break

Installing all necessary libraries...
Installation complete.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted.
API Key loaded successfully.
LLM configured to use Gemini 1.5 Pro.

Loading data from /content/drive/MyDrive/data/output_file.csv into a DataFrame...
Successfully loaded DataFrame with 22352055 rows.

--- Start Chatting With Your Data ---
Ask a question about the ARGO data (or type 'exit' to quit): What was the maximum temperature recorded on January 10, 2010, at a depth of 5 meters?

Creating a fresh Pandas Query Engine...
Thinking...
> Pandas Instructions:
```
```python
max_temp = df.loc[(df['ZAX'] == 5) & (df['TAXIS'] == '2010-01-10'), 'TEMP'].max()
print(max_temp)
```

The maximum temperature recorded on January 10, 2010, at a depth of 5 meters was 25.94.

```
29.959
> Pandas Output: None

Response:
None


Ask a question about the ARGO data (or type 'exit' to quit): exit
Ex